import the basics required packages.

In [3]:
#  Import some data manipulation and plotting packages
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import os

#ignore warnings
import warnings
warnings.filterwarnings("ignore")

from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import SGDRegressor
from sklearn.neighbors import KNeighborsRegressor
from sklearn.tree import DecisionTreeRegressor
from xgboost import XGBRegressor
from sklearn.ensemble import AdaBoostRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.svm import SVR
from sklearn.neural_network import MLPRegressor
from sklearn.metrics import mean_absolute_error,median_absolute_error,mean_squared_log_error, r2_score
from sklearn.model_selection import GridSearchCV

# Read Datasets

In [6]:
# Read all the given datasets into separate dataframes
aquifers = pd.read_csv("Feature-Engineered-Datasets/aquifers_fe.csv")
lakes = pd.read_csv('Feature-Engineered-Datasets/lakes_fe.csv')
rivers = pd.read_csv('Feature-Engineered-Datasets/rivers_fe.csv')
springs = pd.read_csv('Feature-Engineered-Datasets/springs_fe.csv')

# Model

# ==============================================================
# ==============================================================
## ============================= aquifers ====================================
# ==============================================================
# ==============================================================

In [7]:
aquifers.head()

,Mean_Rainfall,Mean_Temp,Actual_Depth,Actual_Volume,Actual_Hydrometry,Date
0,0.415556,6.6250,-6.08026,-8019.271158,-0.083056,1998-01-04
1,2.054444,6.0750,-6.06452,-7956.571285,-0.104167,1998-01-05
2,0.921111,9.0875,-6.15706,-7715.808854,0.011944,1998-01-06
3,0.878889,12.3250,-6.10774,-7731.378766,-0.008611,1998-01-07
4,0.908889,12.6500,-6.05310,-7812.676449,-0.072222,1998-01-08


In [8]:
# Normalizing the columns to remove negative values
aquifers['Mean_Rainfall'] = (aquifers['Mean_Rainfall'] - aquifers['Mean_Rainfall'].min()) / (aquifers['Mean_Rainfall'].max() - aquifers['Mean_Rainfall'].min())
aquifers['Mean_Temp'] = (aquifers['Mean_Temp'] - aquifers['Mean_Temp'].min()) / (aquifers['Mean_Temp'].max() - aquifers['Mean_Temp'].min())
aquifers['Actual_Depth'] = (aquifers['Actual_Depth'] - aquifers['Actual_Depth'].min()) / (aquifers['Actual_Depth'].max() - aquifers['Actual_Depth'].min())
aquifers['Actual_Volume'] = (aquifers['Actual_Volume'] - aquifers['Actual_Volume'].min()) / (aquifers['Actual_Volume'].max() - aquifers['Actual_Volume'].min())
aquifers['Actual_Hydrometry'] = (aquifers['Actual_Hydrometry'] - aquifers['Actual_Hydrometry'].min()) / (aquifers['Actual_Hydrometry'].max() - aquifers['Actual_Hydrometry'].min())

In [9]:
aquifers.head()

,Mean_Rainfall,Mean_Temp,Actual_Depth,Actual_Volume,Actual_Hydrometry,Date
0,0.007267,0.295126,0.984879,0.395764,0.090032,1998-01-04
1,0.035926,0.276216,0.985372,0.401094,0.084072,1998-01-05
2,0.016108,0.379792,0.982474,0.421561,0.116854,1998-01-06
3,0.015369,0.491104,0.984019,0.420238,0.111050,1998-01-07
4,0.015894,0.502278,0.985729,0.413327,0.093091,1998-01-08


In [10]:
# separating the date and target feature
X = aquifers.drop(['Date','Actual_Depth'],axis=1)
y = aquifers['Actual_Depth']

In [11]:
#Dividing the dataset into train and test for features as well as labels
X_train, X_test, y_train, y_test = train_test_split(X,y,test_size=0.2,random_state=42)

In [12]:
#checking the number of rows in training data
len(X_train)

6523

In [14]:

#checking the number of rows in training labels
len(y_train)

6523

In [15]:
#checking the number of rows in testing data
len(X_test)

1631

In [16]:
#checking the number of rows in testing labels
len(y_test)

1631

# ================================================================

### knn

In [17]:
# create a knn regressor
neigh = KNeighborsRegressor()

In [18]:
# find optimal parameters with grid search
param_grid = {
    'n_neighbors': [3,5,7,9,11,19],
    'weights': ['uniform', 'distance'],
    'metric' : ['euclidean','manhattan']
}
grid = GridSearchCV(estimator=neigh, param_grid=param_grid, cv=10,
                    scoring=['neg_median_absolute_error','neg_mean_squared_log_error','r2'], refit = 'neg_mean_squared_log_error',
                    verbose=1, n_jobs=-1)

In [19]:
# fit the grid on the train data
grid_result = grid.fit(X_train, y_train)

Fitting 10 folds for each of 24 candidates, totalling 240 fits


In [20]:
# print the best score and parameters found
print('Best Score: ', grid_result.best_score_)
print('Best Params: ', grid_result.best_params_)

Best Score:  -0.0028144705118891005
Best Params:  {'metric': 'manhattan', 'n_neighbors': 7, 'weights': 'distance'}


In [21]:
# find predictions for test data
y_pred = grid_result.best_estimator_.predict(X_test)

In [22]:
#normalizing y_pred
y_pred = (y_pred - y_pred.min())/(y_pred.max() - y_pred.min())

In [23]:
# find median absolute error
median_absolute_error(y_test,y_pred)

0.02103068051342738

In [24]:
# find root mean square log error
np.sqrt(mean_squared_log_error(y_test,y_pred))

0.05328383316662198

In [25]:
# find r2 score
r2_score(y_test,y_pred)

0.9607730924181563

## ===========================================================================


### Multiple Linear Regression

In [26]:
# create a linear regressor
lr = LinearRegression()

In [27]:
# fit the lr on the train data
lr = lr.fit(X_train, y_train)

In [28]:
# find predictions for test data
y_pred = lr.predict(X_test)

In [29]:
#normalizing y_pred
y_pred = (y_pred - y_pred.min())/(y_pred.max() - y_pred.min())

In [30]:
# find median absolute error
median_absolute_error(y_test,y_pred)

0.3866462761518887

In [31]:
# find root mean square log error
np.sqrt(mean_squared_log_error(y_test,y_pred))

0.31056503793039225

In [32]:
# find r2 score
r2_score(y_test,y_pred)

-0.361099450434454

R2  is negative only when the chosen model does not follow the trend of the data, so fits worse than a horizontal line.

## ==========================================================================================================================================================

### Random Forest

In [ ]:
# create a random forest regressor
rf = RandomForestRegressor()

In [ ]:
# find optimal parameters with grid search
param_grid = {
'bootstrap': [True, False],
 'max_depth': [None,2,10, 20, 50, 100],
 'max_features': ['auto', 'sqrt'],
 'min_samples_leaf': [1, 2, 4],
 'min_samples_split': [2, 5, 10],
 'n_estimators': [100,200,500, 1000]
             }
grid = GridSearchCV(estimator=rf, param_grid=param_grid, cv=5,
                    scoring=['neg_median_absolute_error','neg_mean_squared_log_error','r2'], refit = 'neg_mean_squared_log_error',
                    verbose=1, n_jobs=-1)

In [ ]:
# fit the grid on the train data
grid_result = grid.fit(X_train, y_train)

Fitting 5 folds for each of 864 candidates, totalling 4320 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 2 concurrent workers.
[Parallel(n_jobs=-1)]: Done  46 tasks      | elapsed:  5.2min
[Parallel(n_jobs=-1)]: Done 196 tasks      | elapsed: 21.1min
[Parallel(n_jobs=-1)]: Done 446 tasks      | elapsed: 33.6min
[Parallel(n_jobs=-1)]: Done 796 tasks      | elapsed: 45.0min
[Parallel(n_jobs=-1)]: Done 1246 tasks      | elapsed: 79.3min
[Parallel(n_jobs=-1)]: Done 1796 tasks      | elapsed: 122.7min
[Parallel(n_jobs=-1)]: Done 2446 tasks      | elapsed: 196.9min
[Parallel(n_jobs=-1)]: Done 3196 tasks      | elapsed: 242.8min
[Parallel(n_jobs=-1)]: Done 4046 tasks      | elapsed: 358.7min
[Parallel(n_jobs=-1)]: Done 4320 out of 4320 | elapsed: 392.0min finished


In [ ]:
# print the best score and parameters found
print('Best Score: ', grid_result.best_score_)
print('Best Params: ', grid_result.best_params_)

Best Score:  -0.0021697543201036028
Best Params:  {'bootstrap': True, 'max_depth': None, 'max_features': 'sqrt', 'min_samples_leaf': 1, 'min_samples_split': 2, 'n_estimators': 1000}


In [ ]:
# find predictions for test data
y_pred = grid_result.best_estimator_.predict(X_test)

In [ ]:
#normalizing y_pred
y_pred = (y_pred - y_pred.min())/(y_pred.max() - y_pred.min())

In [ ]:
# find median absolute error
median_absolute_error(y_test,y_pred)

0.019655865584285948

In [ ]:
# find root mean square log error
np.sqrt(mean_squared_log_error(y_test,y_pred))

0.04842439087861878

In [ ]:
# find r2 score
r2_score(y_test,y_pred)

0.9668303549718164

# ===============================================================

### SGDRegression

In [33]:
# create a linear regressor
lr = SGDRegressor()

In [34]:
# find optimal parameters with grid search
param_grid = {
    
    'loss' : ['squared_loss','huber'],
    'alpha': [0.0001,0.001,0.01,0.1],
     'eta0': [0.01,0.1,1,10,100],
    'tol' : [0.00001,0.0001,0.001,0.01,0.1]    
 }
grid = GridSearchCV(estimator=lr, param_grid=param_grid,cv=5,
                    scoring=['neg_median_absolute_error','r2'], refit = 'neg_median_absolute_error',
                    verbose=1, n_jobs=-1)

In [35]:
# fit the grid on the train data
grid_result = grid.fit(X_train, y_train)

Fitting 5 folds for each of 200 candidates, totalling 1000 fits


In [36]:
# print the best score and parameters found
print('Best Score: ', grid_result.best_score_)
print('Best Params: ', grid_result.best_params_)

Best Score:  -0.04661886028567611
Best Params:  {'alpha': 0.01, 'eta0': 0.01, 'loss': 'huber', 'tol': 0.01}


In [37]:
# find predictions for test data
y_pred = grid_result.best_estimator_.predict(X_test)

In [38]:
#normalizing y_pred
y_pred = (y_pred - y_pred.min())/(y_pred.max() - y_pred.min())

In [39]:
# find median absolute error
median_absolute_error(y_test,y_pred)

0.21549702959981842

In [40]:
# find root mean square log error
np.sqrt(mean_squared_log_error(y_test,y_pred))

0.2411878618526684

In [41]:
# find r2 score
r2_score(y_test,y_pred)

0.11108558019503556

# ================================================================ 

### Decision Tree Regressor

In [ ]:
dt = DecisionTreeRegressor()

In [ ]:
# find optimal parameters with grid search
param_grid = {
    
    'criterion' : ['mse','mae'],
    'max_depth': range(10,20),
     'min_samples_split': range(2,10),
    'min_samples_leaf' : range(1,5)    
 }
grid = GridSearchCV(estimator=dt, param_grid=param_grid, cv=10,
                    scoring=['neg_median_absolute_error','neg_mean_squared_log_error','r2'], refit = 'neg_mean_squared_log_error',
                    verbose=1, n_jobs=-1)

In [ ]:
# fit the grid on the train data
grid_result = grid.fit(X_train, y_train)

Fitting 10 folds for each of 640 candidates, totalling 6400 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=-1)]: Done  42 tasks      | elapsed:    1.8s
[Parallel(n_jobs=-1)]: Done 192 tasks      | elapsed:    7.2s
[Parallel(n_jobs=-1)]: Done 442 tasks      | elapsed:   17.7s
[Parallel(n_jobs=-1)]: Done 792 tasks      | elapsed:   32.5s
[Parallel(n_jobs=-1)]: Done 1242 tasks      | elapsed:   53.0s
[Parallel(n_jobs=-1)]: Done 1792 tasks      | elapsed:  1.9min
[Parallel(n_jobs=-1)]: Done 2442 tasks      | elapsed:  3.3min
[Parallel(n_jobs=-1)]: Done 3192 tasks      | elapsed:  4.6min
[Parallel(n_jobs=-1)]: Done 4042 tasks      | elapsed: 40.9min
[Parallel(n_jobs=-1)]: Done 4992 tasks      | elapsed: 64.8min
[Parallel(n_jobs=-1)]: Done 6042 tasks      | elapsed: 90.8min
[Parallel(n_jobs=-1)]: Done 6400 out of 6400 | elapsed: 99.5min finished


In [ ]:
# print the best score and parameters found
print('Best Score: ', grid_result.best_score_)
print('Best Params: ', grid_result.best_params_)

Best Score:  -0.0029725895160034803
Best Params:  {'criterion': 'mae', 'max_depth': 12, 'min_samples_leaf': 2, 'min_samples_split': 4}


In [ ]:
# find predictions for test data
y_pred = grid_result.best_estimator_.predict(X_test)

In [ ]:
#normalizing y_pred
y_pred = (y_pred - y_pred.min())/(y_pred.max() - y_pred.min())

In [ ]:
# find median absolute error
median_absolute_error(y_test,y_pred)

0.01684240996782499

In [ ]:
# find root mean square log error
np.sqrt(mean_squared_log_error(y_test,y_pred))

0.05367391890221074

In [ ]:
# find r2 score
r2_score(y_test,y_pred)

0.9561599730076912

# ================================================================

### XGBoost Regressor

In [ ]:
xgbtree = XGBRegressor()

In [ ]:
# find optimal parameters with grid search
param_grid = {
              'learning_rate': [0.001,0.01,0.1,1],
              'max_depth': range(2,10),
              'subsample':[0.01,0.1,1] ,
              'colsample_bytree': [0.01,0.1,1],
              'n_estimators': [100,200,500,1000]}
grid = GridSearchCV(estimator=xgbtree, param_grid=param_grid, cv=5,
                    scoring=['neg_median_absolute_error','r2'], refit = 'neg_median_absolute_error',
                    verbose=1, n_jobs=-1)

In [ ]:
# fit the grid on the train data
grid_result = grid.fit(X_train, y_train)

Fitting 5 folds for each of 1152 candidates, totalling 5760 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 2 concurrent workers.
[Parallel(n_jobs=-1)]: Done  46 tasks      | elapsed:   14.6s
[Parallel(n_jobs=-1)]: Done 196 tasks      | elapsed:  1.6min
[Parallel(n_jobs=-1)]: Done 446 tasks      | elapsed:  4.6min
[Parallel(n_jobs=-1)]: Done 796 tasks      | elapsed:  8.3min
[Parallel(n_jobs=-1)]: Done 1246 tasks      | elapsed: 13.4min
[Parallel(n_jobs=-1)]: Done 1796 tasks      | elapsed: 19.9min
[Parallel(n_jobs=-1)]: Done 2446 tasks      | elapsed: 27.1min
[Parallel(n_jobs=-1)]: Done 3196 tasks      | elapsed: 35.3min
[Parallel(n_jobs=-1)]: Done 4046 tasks      | elapsed: 45.0min
[Parallel(n_jobs=-1)]: Done 4996 tasks      | elapsed: 60.7min
[Parallel(n_jobs=-1)]: Done 5760 out of 5760 | elapsed: 74.4min finished


[12:42:25] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.


In [ ]:
# print the best score and parameters found
print('Best Score: ', grid_result.best_score_)
print('Best Params: ', grid_result.best_params_)

Best Score:  -0.016081142855098807
Best Params:  {'colsample_bytree': 1, 'learning_rate': 0.01, 'max_depth': 8, 'n_estimators': 1000, 'subsample': 1}


In [ ]:
# find predictions for test data
y_pred = grid_result.best_estimator_.predict(X_test)

In [ ]:
#normalizing y_pred
y_pred = (y_pred - y_pred.min())/(y_pred.max() - y_pred.min())

In [ ]:
# find median absolute error
median_absolute_error(y_test,y_pred)

0.02528232677151125

In [ ]:
# find root mean square log error
np.sqrt(mean_squared_log_error(y_test,y_pred))

0.053061569375817304

In [ ]:
# find r2 score
r2_score(y_test,y_pred)

0.9585200503118089

# ================================================================

### AdaBoost Regressor

In [42]:
abreg = AdaBoostRegressor(base_estimator=DecisionTreeRegressor(),
                              random_state=0)

In [43]:
# find optimal parameters with grid search
param_grid = { 'loss' : ['linear', 'square', 'exponential'],
              'learning_rate': [0.001,0.01,0.1,1],
              'base_estimator__max_depth': range(2,10),
              'n_estimators': [100,200,500,1000]}
grid = GridSearchCV(estimator=abreg, param_grid=param_grid, cv=5,
                    scoring=['neg_median_absolute_error','r2'], refit = 'neg_median_absolute_error',
                    verbose=1, n_jobs=-1)

In [ ]:
# fit the grid on the train data
grid_result = grid.fit(X_train, y_train)

Fitting 5 folds for each of 384 candidates, totalling 1920 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 2 concurrent workers.
[Parallel(n_jobs=-1)]: Done  46 tasks      | elapsed:  1.8min
[Parallel(n_jobs=-1)]: Done 196 tasks      | elapsed:  5.8min
[Parallel(n_jobs=-1)]: Done 446 tasks      | elapsed: 12.6min
[Parallel(n_jobs=-1)]: Done 796 tasks      | elapsed: 27.6min
[Parallel(n_jobs=-1)]: Done 1246 tasks      | elapsed: 53.7min
[Parallel(n_jobs=-1)]: Done 1796 tasks      | elapsed: 100.0min
[Parallel(n_jobs=-1)]: Done 1920 out of 1920 | elapsed: 108.9min finished


In [ ]:
# print the best score and parameters found
print('Best Score: ', grid_result.best_score_)
print('Best Params: ', grid_result.best_params_)

Best Score:  -0.014184367982375096
Best Params:  {'base_estimator__max_depth': 9, 'learning_rate': 1, 'loss': 'square', 'n_estimators': 200}


In [ ]:
# find predictions for test data
y_pred = grid_result.best_estimator_.predict(X_test)

In [ ]:
#normalizing y_pred
y_pred = (y_pred - y_pred.min())/(y_pred.max() - y_pred.min())

In [ ]:
# find median absolute error
median_absolute_error(y_test,y_pred)

0.02192634636241439

In [ ]:
# find root mean square log error
np.sqrt(mean_squared_log_error(y_test,y_pred))

0.051313886313701484

In [ ]:
# find r2 score
r2_score(y_test,y_pred)

0.9641048020090041

# ===============================================================

### MLP Regressor

In [44]:
from tensorflow.keras.callbacks import ModelCheckpoint, TensorBoard

In [ ]:
import datetime

In [46]:
# Creating a Neural Network Model
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Activation
from tensorflow.keras.optimizers import Adam

In [47]:
model = Sequential()
model.add(Dense(1000,activation='relu'))
model.add(Dense(1000,activation='relu'))
model.add(Dense(1000,activation='relu'))
model.add(Dense(1000,activation='relu'))
model.add(Dense(1))

In [48]:
import tensorflow.keras.backend as K

In [49]:

def rmsle(y_true, y_pred):
    msle = tf.keras.losses.MeanSquaredLogarithmicError()
    return K.sqrt(msle(y_true, y_pred)) 

In [50]:
model.compile(optimizer='Adam',loss=rmsle,metrics=[tf.keras.metrics.RootMeanSquaredError()])

In [52]:
model.fit(x=X_train,y=y_train,
          validation_data=(X_test,y_test),
          
          batch_size=128,epochs=100)


Epoch 1/100
51/51 [==============================] - 6s 85ms/step - loss: 0.2137 - root_mean_squared_error: 0.3198 - val_loss: 0.1086 - val_root_mean_squared_error: 0.1742
Epoch 2/100
51/51 [==============================] - 4s 77ms/step - loss: 0.1004 - root_mean_squared_error: 0.1555 - val_loss: 0.0872 - val_root_mean_squared_error: 0.1367
Epoch 3/100
51/51 [==============================] - 4s 78ms/step - loss: 0.0839 - root_mean_squared_error: 0.1285 - val_loss: 0.0809 - val_root_mean_squared_error: 0.1232
Epoch 4/100
51/51 [==============================] - 4s 72ms/step - loss: 0.0799 - root_mean_squared_error: 0.1246 - val_loss: 0.0728 - val_root_mean_squared_error: 0.1112
Epoch 5/100
51/51 [==============================] - 4s 74ms/step - loss: 0.0701 - root_mean_squared_error: 0.1161 - val_loss: 0.0661 - val_root_mean_squared_error: 0.1038
Epoch 6/100
51/51 [==============================] - 4s 73ms/step - loss: 0.0686 - root_mean_squared_error: 0.1087 - val_loss: 0.0651 - val_

51/51 [==============================] - 3s 64ms/step - loss: 0.0488 - root_mean_squared_error: 0.0771 - val_loss: 0.0443 - val_root_mean_squared_error: 0.0688
Epoch 96/100
51/51 [==============================] - 3s 68ms/step - loss: 0.0416 - root_mean_squared_error: 0.0651 - val_loss: 0.0436 - val_root_mean_squared_error: 0.0668
Epoch 97/100
51/51 [==============================] - 4s 77ms/step - loss: 0.0409 - root_mean_squared_error: 0.0613 - val_loss: 0.0522 - val_root_mean_squared_error: 0.0788
Epoch 98/100
51/51 [==============================] - 4s 81ms/step - loss: 0.0415 - root_mean_squared_error: 0.0620 - val_loss: 0.0504 - val_root_mean_squared_error: 0.0763
Epoch 99/100
51/51 [==============================] - 5s 98ms/step - loss: 0.0427 - root_mean_squared_error: 0.0654 - val_loss: 0.0531 - val_root_mean_squared_error: 0.0871
Epoch 100/100
51/51 [==============================] - 4s 71ms/step - loss: 0.0497 - root_mean_squared_error: 0.0766 - val_loss: 0.0444 - val_root_m

In [ ]:
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense (Dense)                (None, 1000)              5000      
_________________________________________________________________
dense_1 (Dense)              (None, 1000)              1001000   
_________________________________________________________________
dense_2 (Dense)              (None, 1000)              1001000   
_________________________________________________________________
dense_3 (Dense)              (None, 1000)              1001000   
_________________________________________________________________
dense_4 (Dense)              (None, 1)                 1001      
Total params: 3,009,001
Trainable params: 3,009,001
Non-trainable params: 0
_________________________________________________________________


In [53]:
K.clear_session()

In [54]:
y_pred = model.predict(X_test)

In [55]:
# find median absolute error
median_absolute_error(y_test,y_pred)

0.019173547932702206

In [56]:
# find root mean square log error
np.sqrt(mean_squared_log_error(y_test,y_pred))

0.047100793422137466

In [57]:
# find r2 score
r2_score(y_test,y_pred)

0.9680868404916957

# =============================================================
# =============================================================
## ============================= Lakes =====================================
# =============================================================
# =============================================================

In [58]:
lakes.head()

,Date,Mean_Rainfall,Mean_Temp,Actual_Flow_Rate,Actual_Lake_Level
0,0,2.857312,14.530141,2.578255,249.606745
1,1,2.857312,14.530141,2.578255,249.606745
2,2,2.857312,14.530141,2.578255,249.606745
3,3,2.857312,14.530141,2.578255,249.606745
4,4,2.857312,14.530141,2.578255,249.606745


In [59]:
# Normalizing the columns to remove negative values
lakes['Mean_Rainfall'] = (lakes['Mean_Rainfall'] - lakes['Mean_Rainfall'].min()) / (lakes['Mean_Rainfall'].max() - lakes['Mean_Rainfall'].min())
lakes['Mean_Temp'] = (lakes['Mean_Temp'] - lakes['Mean_Temp'].min()) / (lakes['Mean_Temp'].max() - lakes['Mean_Temp'].min())
lakes['Actual_Flow_Rate'] = (lakes['Actual_Flow_Rate'] - lakes['Actual_Flow_Rate'].min()) / (lakes['Actual_Flow_Rate'].max() - lakes['Actual_Flow_Rate'].min())
lakes['Actual_Lake_Level'] = (lakes['Actual_Lake_Level'] - lakes['Actual_Lake_Level'].min()) / (lakes['Actual_Lake_Level'].max() - lakes['Actual_Lake_Level'].min())

In [60]:
lakes.head()

,Date,Mean_Rainfall,Mean_Temp,Actual_Flow_Rate,Actual_Lake_Level
0,0,0.033568,0.505213,0.028683,0.658369
1,1,0.033568,0.505213,0.028683,0.658369
2,2,0.033568,0.505213,0.028683,0.658369
3,3,0.033568,0.505213,0.028683,0.658369
4,4,0.033568,0.505213,0.028683,0.658369


In [61]:
# separating the date and target feature
X = lakes.drop(['Date','Actual_Flow_Rate','Actual_Lake_Level'],axis=1)
y = lakes[['Actual_Flow_Rate','Actual_Lake_Level']]

In [62]:
#Dividing the dataset into train and test for features as well as labels
X_train, X_test, y_train, y_test = train_test_split(X,y,test_size=0.2,random_state=42)

In [63]:
#checking the number of rows in training data
len(X_train)

5282

In [64]:
#checking the number of rows in training labels
len(y_train)

5282

In [65]:
#checking the number of rows in testing data
len(X_test)

1321

In [66]:
#checking the number of rows in testing labels
len(y_test)

1321

# ================================================================

### knn

In [67]:
# create a knn regressor
neigh = KNeighborsRegressor()

In [68]:
# find optimal parameters with grid search
param_grid = {
    'n_neighbors': [3,5,7,9,11,19],
    'weights': ['uniform', 'distance'],
    'metric' : ['euclidean','manhattan']
}
grid = GridSearchCV(estimator=neigh, param_grid=param_grid,
                    scoring=['neg_median_absolute_error','neg_mean_squared_log_error','r2'], refit = 'neg_mean_squared_log_error',
                    verbose=1, n_jobs=-1)

In [69]:
# fit the grid on the train data
grid_result = grid.fit(X_train, y_train)

Fitting 5 folds for each of 24 candidates, totalling 120 fits


In [70]:
# print the best score and parameters found
print('Best Score: ', grid_result.best_score_)
print('Best Params: ', grid_result.best_params_)

Best Score:  -0.007168036852265819
Best Params:  {'metric': 'euclidean', 'n_neighbors': 19, 'weights': 'uniform'}


In [71]:
# find predictions for test data
y_pred = grid_result.best_estimator_.predict(X_test)

In [72]:
# normalizing the predicted values
y_pred[:,0] = (y_pred[:,0] - y_pred[:,0].min())/(y_pred[:,0].max() - y_pred[:,0].min())

In [73]:

# normalizing the predicted values
y_pred[:,1] = (y_pred[:,1] - y_pred[:,1].min())/(y_pred[:,1].max() - y_pred[:,1].min())

In [74]:
# find median absolute error
median_absolute_error(y_test.iloc[:,0],y_pred[:,0])

0.11279539978027128

In [75]:
# find root mean square log error
np.sqrt(mean_squared_log_error(y_test.iloc[:,0],y_pred[:,0]))

0.1570520689352546

In [76]:
# find r2 score
r2_score(y_test.iloc[:,0],y_pred[:,0])

-18.42880247998059

#### the secoond prediction

In [77]:
# find median absolute error
median_absolute_error(y_test.iloc[:,1],y_pred[:,1])

0.1811120486734537

In [78]:
# find root mean square log error
np.sqrt(mean_squared_log_error(y_test.iloc[:,1],y_pred[:,1]))

0.1791217384377378

In [79]:
# find r2 score
r2_score(y_test.iloc[:,1],y_pred[:,1])

-1.4160763701487715

# ================================================================

### Multiple Linear Regression

In [80]:
# create a linear regressor
lr = LinearRegression()

In [81]:
# fit the lr on train data
lr = lr.fit(X_train, y_train)

In [82]:
# find the predictions on test data
y_pred = lr.predict(X_test)

In [83]:
# normalize the predictions
y_pred[:,0] = (y_pred[:,0] - y_pred[:,0].min())/(y_pred[:,0].max() - y_pred[:,0].min())

In [84]:
# normalize the predictions
y_pred[:,1] = (y_pred[:,1] - y_pred[:,1].min())/(y_pred[:,1].max() - y_pred[:,1].min())

In [85]:
# find median absolute error
median_absolute_error(y_test.iloc[:,0],y_pred[:,0])

0.5015375144710312

In [86]:
# find root mean square log error
np.sqrt(mean_squared_log_error(y_test.iloc[:,0],y_pred[:,0]))

0.4073832306770018

In [87]:
# find r2 score
r2_score(y_test.iloc[:,0],y_pred[:,0])

-156.95424008399553

####  the second prediction

In [88]:
# find median absolute error
median_absolute_error(y_test.iloc[:,1],y_pred[:,1])

0.16368554371553756

In [89]:
# find root mean square log error
np.sqrt(mean_squared_log_error(y_test.iloc[:,1],y_pred[:,1]))

0.14938090265660803

In [90]:
# find r2 score
r2_score(y_test.iloc[:,1],y_pred[:,1])

-0.8211047468624715

R2  is negative only when the chosen model does not follow the trend of the data, so fits worse than a horizontal line.

# ================================================================

### Random Forest

In [91]:
# create a random forest regressor
rf = RandomForestRegressor()

In [92]:
# find optimal parameters with grid search
param_grid = {
'bootstrap': [True, False],
 'max_depth': [None,10, 20, 50, 100],
 'max_features': ['auto', 'sqrt'],
 'min_samples_leaf': [1, 2, 4],
 'min_samples_split': [2, 5, 10],
 'n_estimators': [100,200,500, 1000]
             }
grid = GridSearchCV(estimator=rf, param_grid=param_grid,
                    scoring=['neg_median_absolute_error','neg_mean_squared_log_error','r2'], refit = 'neg_mean_squared_log_error',
                    verbose=1, n_jobs=-1)

In [ ]:
# fit the grid on the train data
grid_result = grid.fit(X_train, y_train)

Fitting 5 folds for each of 720 candidates, totalling 3600 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 2 concurrent workers.
[Parallel(n_jobs=-1)]: Done  46 tasks      | elapsed:  1.5min
[Parallel(n_jobs=-1)]: Done 196 tasks      | elapsed:  6.2min
[Parallel(n_jobs=-1)]: Done 446 tasks      | elapsed: 12.5min
[Parallel(n_jobs=-1)]: Done 796 tasks      | elapsed: 21.3min
[Parallel(n_jobs=-1)]: Done 1246 tasks      | elapsed: 34.7min
[Parallel(n_jobs=-1)]: Done 1796 tasks      | elapsed: 50.6min
[Parallel(n_jobs=-1)]: Done 2446 tasks      | elapsed: 73.4min
[Parallel(n_jobs=-1)]: Done 3196 tasks      | elapsed: 101.1min
[Parallel(n_jobs=-1)]: Done 3600 out of 3600 | elapsed: 116.7min finished


In [ ]:
# print the best score and parameters found
print('Best Score: ', grid_result.best_score_)
print('Best Params: ', grid_result.best_params_)

Best Score:  -0.00704013646129713
Best Params:  {'bootstrap': True, 'max_depth': 10, 'max_features': 'sqrt', 'min_samples_leaf': 2, 'min_samples_split': 10, 'n_estimators': 500}


In [ ]:
# find predictions for test data
y_pred = grid_result.best_estimator_.predict(X_test)

In [ ]:
# normalize the predictions
y_pred[:,0] = (y_pred[:,0] - y_pred[:,0].min())/(y_pred[:,0].max() - y_pred[:,0].min())

In [ ]:
# normalize the predictions
y_pred[:,1] = (y_pred[:,1] - y_pred[:,1].min())/(y_pred[:,1].max() - y_pred[:,1].min())

In [ ]:
# find median absolute error
median_absolute_error(y_test.iloc[:,0],y_pred[:,0])

0.13486025796631473

In [ ]:
# find root mean square log error
np.sqrt(mean_squared_log_error(y_test.iloc[:,0],y_pred[:,0]))

0.18098893002166355

In [ ]:
# find r2 score
r2_score(y_test.iloc[:,0],y_pred[:,0])

-25.890108945315262

#### the second prediction

In [ ]:
# find median absolute error
median_absolute_error(y_test.iloc[:,1],y_pred[:,1])

0.13719308429587834

In [ ]:
# find root mean square log error
np.sqrt(mean_squared_log_error(y_test.iloc[:,1],y_pred[:,1]))

0.1461849494815439

In [ ]:
# find r2 score
r2_score(y_test.iloc[:,1],y_pred[:,1])

-0.6840530272584708

# ================================================================

### SGRegression

In [93]:
from sklearn.multioutput import MultiOutputRegressor

In [94]:
# create a linear regressor
lr = MultiOutputRegressor(SGDRegressor())

In [95]:
# find optimal parameters with grid search
param_grid = {
    
    'estimator__loss' : ['squared_loss','huber'],
    'estimator__alpha': [0.0001,0.001,0.01,0.1],
     'estimator__eta0': [0.01,0.1,1,10,100],
    'estimator__tol' : [0.00001,0.0001,0.001,0.01,0.1]    
 }
grid = GridSearchCV(estimator=lr, param_grid=param_grid,
                    scoring=['neg_median_absolute_error','r2'], refit = 'neg_median_absolute_error',
                    verbose=1, n_jobs=-1)

In [96]:
# fit the grid on the train data
grid_result = grid.fit(X_train, y_train)

Fitting 5 folds for each of 200 candidates, totalling 1000 fits


In [97]:
# print the best score and parameters found
print('Best Score: ', grid_result.best_score_)
print('Best Params: ', grid_result.best_params_)

Best Score:  -0.0499613785813279
Best Params:  {'estimator__alpha': 0.01, 'estimator__eta0': 1, 'estimator__loss': 'huber', 'estimator__tol': 1e-05}


In [98]:
# find predictions for test data
y_pred = grid_result.best_estimator_.predict(X_test)

In [99]:
#normalizing y_pred
y_pred = (y_pred - y_pred.min())/(y_pred.max() - y_pred.min())

In [100]:
# find median absolute error
median_absolute_error(y_test,y_pred)

0.11297241870299786

In [101]:
# find root mean square log error
np.sqrt(mean_squared_log_error(y_test,y_pred))

0.12222366848096275

In [102]:
# find r2 score
r2_score(y_test,y_pred)

-0.8425463518399292

# ================================================================

### Decision Tree Regressor

In [ ]:
dt = DecisionTreeRegressor()

In [ ]:
# find optimal parameters with grid search
param_grid = {
    
    'criterion' : ['mse','mae'],
    'max_depth': range(10,20),
     'min_samples_split': range(2,10),
    'min_samples_leaf' : range(1,5)    
 }
grid = GridSearchCV(estimator=dt, param_grid=param_grid, cv=10,
                    scoring=['neg_median_absolute_error','neg_mean_squared_log_error','r2'], refit = 'neg_mean_squared_log_error',
                    verbose=1, n_jobs=-1)

In [ ]:
# fit the grid on the train data
grid_result = grid.fit(X_train, y_train)

Fitting 10 folds for each of 640 candidates, totalling 6400 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 2 concurrent workers.
[Parallel(n_jobs=-1)]: Done 202 tasks      | elapsed:    3.5s
[Parallel(n_jobs=-1)]: Done 1402 tasks      | elapsed:   16.4s
[Parallel(n_jobs=-1)]: Done 3262 tasks      | elapsed:   58.4s
[Parallel(n_jobs=-1)]: Done 3612 tasks      | elapsed:  3.0min
[Parallel(n_jobs=-1)]: Done 4062 tasks      | elapsed:  5.6min
[Parallel(n_jobs=-1)]: Done 4612 tasks      | elapsed:  8.9min
[Parallel(n_jobs=-1)]: Done 5262 tasks      | elapsed: 12.8min
[Parallel(n_jobs=-1)]: Done 6012 tasks      | elapsed: 17.3min
[Parallel(n_jobs=-1)]: Done 6400 out of 6400 | elapsed: 19.7min finished


In [ ]:
# print the best score and parameters found
print('Best Score: ', grid_result.best_score_)
print('Best Params: ', grid_result.best_params_)

Best Score:  -0.00773425390770469
Best Params:  {'criterion': 'mse', 'max_depth': 10, 'min_samples_leaf': 4, 'min_samples_split': 9}


In [ ]:
# find predictions for test data
y_pred = grid_result.best_estimator_.predict(X_test)

In [ ]:
#normalizing y_pred
y_pred = (y_pred - y_pred.min())/(y_pred.max() - y_pred.min())

In [ ]:
# find median absolute error
median_absolute_error(y_test,y_pred)

0.051486780984233205

In [ ]:
# find root mean square log error
np.sqrt(mean_squared_log_error(y_test,y_pred))

0.08403082471073176

In [ ]:
# find r2 score
r2_score(y_test,y_pred)

-0.091339400115092

# ================================================================

### XGBoost Regressor

In [ ]:
from sklearn.multioutput import MultiOutputRegressor

In [ ]:
xgbtree = MultiOutputRegressor(XGBRegressor())

In [ ]:
# find optimal parameters with grid search
param_grid = {
              'estimator__learning_rate': [0.001,0.01,0.1,1],
              'estimator__max_depth': range(2,10),
              'estimator__subsample':[0.01,0.1,1] ,
              'estimator__colsample_bytree': [0.01,0.1,1],
              'estimator__n_estimators': [100,200,500,1000]}
grid = GridSearchCV(estimator=xgbtree, param_grid=param_grid, cv=5,
                    scoring=['neg_median_absolute_error','r2'], refit = 'neg_median_absolute_error',
                    verbose=1, n_jobs=-1)

In [ ]:
# fit the grid on the train data
grid_result = grid.fit(X_train, y_train)

Fitting 5 folds for each of 1152 candidates, totalling 5760 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 2 concurrent workers.
[Parallel(n_jobs=-1)]: Done  46 tasks      | elapsed:   20.7s
[Parallel(n_jobs=-1)]: Done 196 tasks      | elapsed:  2.2min
[Parallel(n_jobs=-1)]: Done 446 tasks      | elapsed:  5.7min
[Parallel(n_jobs=-1)]: Done 796 tasks      | elapsed: 11.1min
[Parallel(n_jobs=-1)]: Done 1246 tasks      | elapsed: 18.8min
[Parallel(n_jobs=-1)]: Done 1796 tasks      | elapsed: 28.8min
[Parallel(n_jobs=-1)]: Done 2446 tasks      | elapsed: 38.1min
[Parallel(n_jobs=-1)]: Done 3196 tasks      | elapsed: 50.5min
[Parallel(n_jobs=-1)]: Done 4046 tasks      | elapsed: 64.6min
[Parallel(n_jobs=-1)]: Done 4996 tasks      | elapsed: 82.9min
[Parallel(n_jobs=-1)]: Done 5760 out of 5760 | elapsed: 99.7min finished


[20:25:02] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[20:25:03] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.


In [ ]:
# print the best score and parameters found
print('Best Score: ', grid_result.best_score_)
print('Best Params: ', grid_result.best_params_)

Best Score:  -0.05736183594793019
Best Params:  {'estimator__colsample_bytree': 0.01, 'estimator__learning_rate': 0.01, 'estimator__max_depth': 8, 'estimator__n_estimators': 1000, 'estimator__subsample': 0.01}


In [ ]:
# find predictions for test data
y_pred = grid_result.best_estimator_.predict(X_test)

In [ ]:
#normalizing y_pred
y_pred = (y_pred - y_pred.min())/(y_pred.max() - y_pred.min())

In [ ]:
# find median absolute error
median_absolute_error(y_test,y_pred)

0.073711019955124

In [ ]:
# find root mean square log error
np.sqrt(mean_squared_log_error(y_test,y_pred))

0.0992570475717765

In [ ]:
# find r2 score
r2_score(y_test,y_pred)

-0.3250485495553004

# ================================================================

### AdaBoost Regressor

In [ ]:
abreg = MultiOutputRegressor(AdaBoostRegressor(base_estimator=DecisionTreeRegressor(),
                              random_state=0))

In [ ]:
# find optimal parameters with grid search
param_grid = { 'estimator__loss' : ['linear', 'square', 'exponential'],
              'estimator__learning_rate': [0.001,0.01,0.1,1],
              'estimator__base_estimator__max_depth': range(2,10),
              'estimator__n_estimators': [100,200,500,1000]}
grid = GridSearchCV(estimator=abreg, param_grid=param_grid, cv=5,
                    scoring=['neg_median_absolute_error','r2'], refit = 'neg_median_absolute_error',
                    verbose=1, n_jobs=-1)

In [ ]:
# fit the grid on the train data
grid_result = grid.fit(X_train, y_train)

Fitting 5 folds for each of 384 candidates, totalling 1920 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 2 concurrent workers.
[Parallel(n_jobs=-1)]: Done  46 tasks      | elapsed:  1.9min
[Parallel(n_jobs=-1)]: Done 196 tasks      | elapsed:  5.9min
[Parallel(n_jobs=-1)]: Done 446 tasks      | elapsed: 13.7min
[Parallel(n_jobs=-1)]: Done 796 tasks      | elapsed: 28.4min
[Parallel(n_jobs=-1)]: Done 1246 tasks      | elapsed: 48.8min
[Parallel(n_jobs=-1)]: Done 1796 tasks      | elapsed: 80.0min
[Parallel(n_jobs=-1)]: Done 1920 out of 1920 | elapsed: 84.8min finished


In [ ]:
# print the best score and parameters found
print('Best Score: ', grid_result.best_score_)
print('Best Params: ', grid_result.best_params_)

Best Score:  -0.05696000343719722
Best Params:  {'estimator__base_estimator__max_depth': 9, 'estimator__learning_rate': 0.001, 'estimator__loss': 'exponential', 'estimator__n_estimators': 100}


In [ ]:
# find predictions for test data
y_pred = grid_result.best_estimator_.predict(X_test)

In [ ]:
#normalizing y_pred
y_pred = (y_pred - y_pred.min())/(y_pred.max() - y_pred.min())

In [ ]:
# find median absolute error
median_absolute_error(y_test,y_pred)

0.049232930801555316

In [ ]:
# find root mean square log error
np.sqrt(mean_squared_log_error(y_test,y_pred))

0.08178335355032659

In [ ]:
# find r2 score
r2_score(y_test,y_pred)

-0.053764952172182356

# ================================================================

### MLP Regressor

In [104]:
from tensorflow.keras.callbacks import ModelCheckpoint, TensorBoard

In [105]:
import datetime

In [106]:
# Creating a Neural Network Model
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Activation, Dropout
from tensorflow.keras.optimizers import Adam

In [107]:
model = Sequential()
model.add(Dense(1024,activation='relu'))
model.add(Dropout(0.2))
model.add(Dense(1024,activation='relu'))
#model.add(Dropout(0.2))
model.add(Dense(128,activation='relu'))
#model.add(Dropout(0.2))
model.add(Dense(128,activation='relu'))
#model.add(Dropout(0.2))
model.add(Dense(128,activation='relu'))
model.add(Dense(2))

In [108]:
import tensorflow.keras.backend as K

In [109]:

def rmsle(y_true, y_pred):
    msle = tf.keras.losses.MeanSquaredLogarithmicError()
    return K.sqrt(msle(y_true, y_pred)) 

In [110]:
model.compile(optimizer='Adam',loss=rmsle,metrics=[tf.keras.metrics.RootMeanSquaredError()])

In [111]:
model.fit(x=X_train,y=y_train,
          validation_data=(X_test,y_test),
          batch_size=64,epochs=100)


Epoch 1/100
83/83 [==============================] - 4s 36ms/step - loss: 0.1525 - root_mean_squared_error: 0.2419 - val_loss: 0.0762 - val_root_mean_squared_error: 0.1155s: 0.1615 - root_mean_squared_error
Epoch 2/100
83/83 [==============================] - 3s 31ms/step - loss: 0.0846 - root_mean_squared_error: 0.1269 - val_loss: 0.0765 - val_root_mean_squared_error: 0.1169
Epoch 3/100
83/83 [==============================] - 2s 29ms/step - loss: 0.0852 - root_mean_squared_error: 0.1275 - val_loss: 0.0793 - val_root_mean_squared_error: 0.1198
Epoch 4/100
83/83 [==============================] - 3s 34ms/step - loss: 0.0833 - root_mean_squared_error: 0.1246 - val_loss: 0.0784 - val_root_mean_squared_error: 0.1183
Epoch 5/100
83/83 [==============================] - 2s 30ms/step - loss: 0.0832 - root_mean_squared_error: 0.1248 - val_loss: 0.0777 - val_root_mean_squared_error: 0.1167
Epoch 6/100
83/83 [==============================] - 2s 25ms/step - loss: 0.0829 - root_mean_squared_erro

Epoch 48/100
83/83 [==============================] - 3s 37ms/step - loss: 0.0826 - root_mean_squared_error: 0.1233 - val_loss: 0.0755 - val_root_mean_squared_error: 0.1140
Epoch 49/100
83/83 [==============================] - 3s 36ms/step - loss: 0.0820 - root_mean_squared_error: 0.1228 - val_loss: 0.0755 - val_root_mean_squared_error: 0.1140
Epoch 50/100
83/83 [==============================] - 3s 34ms/step - loss: 0.0841 - root_mean_squared_error: 0.1257 - val_loss: 0.0756 - val_root_mean_squared_error: 0.1142
Epoch 51/100
83/83 [==============================] - 3s 33ms/step - loss: 0.0821 - root_mean_squared_error: 0.1231 - val_loss: 0.0755 - val_root_mean_squared_error: 0.1142
Epoch 52/100
83/83 [==============================] - 3s 33ms/step - loss: 0.0823 - root_mean_squared_error: 0.1235 - val_loss: 0.0758 - val_root_mean_squared_error: 0.1154
Epoch 53/100
83/83 [==============================] - 3s 31ms/step - loss: 0.0839 - root_mean_squared_error: 0.1252 - val_loss: 0.0765 

In [112]:
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense (Dense)                (None, 1024)              3072      
_________________________________________________________________
dropout (Dropout)            (None, 1024)              0         
_________________________________________________________________
dense_1 (Dense)              (None, 1024)              1049600   
_________________________________________________________________
dense_2 (Dense)              (None, 128)               131200    
_________________________________________________________________
dense_3 (Dense)              (None, 128)               16512     
_________________________________________________________________
dense_4 (Dense)              (None, 128)               16512     
_________________________________________________________________
dense_5 (Dense)              (None, 2)                 2

In [113]:
K.clear_session()

In [114]:
y_pred = model.predict(X_test)

In [115]:
y_pred.shape

(1321, 2)

In [116]:
# find median absolute error
median_absolute_error(y_test,y_pred)

0.05627708154857054

In [117]:
# find root mean square log error
np.sqrt(mean_squared_log_error(y_test,y_pred))

0.07571663797547266

In [118]:
# find r2 score
r2_score(y_test,y_pred)

0.08660359814715346

# ================================================================

# =============================================================
# =============================================================
## ============================= Rivers =====================================
# =============================================================
# =============================================================

In [119]:
rivers.head()

,Date,Mean_Rainfall,Mean_Temp,Actual_Hydrometry
0,0,2.789049,16.605386,1.602951
1,1,2.789049,16.605386,1.602951
2,2,2.789049,16.605386,1.602951
3,3,2.789049,16.605386,1.602951
4,4,2.789049,16.605386,1.602951


In [120]:
# Normalizing the columns to remove negative values
rivers['Mean_Rainfall'] = (rivers['Mean_Rainfall'] - rivers['Mean_Rainfall'].min()) / (rivers['Mean_Rainfall'].max() - rivers['Mean_Rainfall'].min())
rivers['Mean_Temp'] = (rivers['Mean_Temp'] - rivers['Mean_Temp'].min()) / (rivers['Mean_Temp'].max() - rivers['Mean_Temp'].min())
rivers['Actual_Hydrometry'] = (rivers['Actual_Hydrometry'] - rivers['Actual_Hydrometry'].min()) / (rivers['Actual_Hydrometry'].max() - rivers['Actual_Hydrometry'].min())

In [121]:
rivers.head()

,Date,Mean_Rainfall,Mean_Temp,Actual_Hydrometry
0,0,0.060726,0.513441,0.260642
1,1,0.060726,0.513441,0.260642
2,2,0.060726,0.513441,0.260642
3,3,0.060726,0.513441,0.260642
4,4,0.060726,0.513441,0.260642


In [122]:
# separating the date and target feature
X = rivers.drop(['Date','Actual_Hydrometry'],axis=1)
y = rivers['Actual_Hydrometry']

In [123]:
#Dividing the dataset into train and test for features as well as labels
X_train, X_test, y_train, y_test = train_test_split(X,y,test_size=0.2,random_state=42)

In [124]:
#checking the number of rows in training data
len(X_train)

6573

In [125]:
#checking the number of rows in training labels
len(y_train)

6573

In [126]:
#checking the number of rows in testing data
len(X_test)

1644

In [127]:
#checking the number of rows in testing labels
len(y_test)

1644

# ===============================================================

### knn

In [128]:
# create a knn regressor
neigh = KNeighborsRegressor()

In [129]:
# find optimal parameters with grid search
param_grid = {
    'n_neighbors': [3,5,7,9,11,19],
    'weights': ['uniform', 'distance'],
    'metric' : ['euclidean','manhattan']
}
grid = GridSearchCV(estimator=neigh, param_grid=param_grid,
                    scoring=['neg_median_absolute_error','neg_mean_squared_log_error','r2'], refit = 'neg_mean_squared_log_error',
                    verbose=1, n_jobs=-1)

In [130]:
# fit the grid on the train data
grid_result = grid.fit(X_train, y_train)

Fitting 5 folds for each of 24 candidates, totalling 120 fits


In [131]:
# print the best score and parameters found
print('Best Score: ', grid_result.best_score_)
print('Best Params: ', grid_result.best_params_)

Best Score:  -0.0044697518815531015
Best Params:  {'metric': 'euclidean', 'n_neighbors': 19, 'weights': 'uniform'}


In [132]:
# find predictions for test data
y_pred = grid_result.best_estimator_.predict(X_test)

In [133]:
#normalizing y_pred
y_pred = (y_pred - y_pred.min())/(y_pred.max() - y_pred.min())

In [134]:
# find median absolute error
median_absolute_error(y_test,y_pred)

0.19775886224766392

In [135]:
# find root mean square log error
np.sqrt(mean_squared_log_error(y_test,y_pred))

0.1810277615039372

In [136]:
# find r2 score
r2_score(y_test,y_pred)

-6.605017025038958

# ===============================================================

### Multiple Linear Regression

In [137]:
# create a linear regressor
lr = LinearRegression()

In [138]:
# fit the lr on the train data
lr = lr.fit(X_train, y_train)

In [139]:
# find the predictions
y_pred = lr.predict(X_test)

In [140]:
#normalizing y_pred
y_pred = (y_pred - y_pred.min())/(y_pred.max() - y_pred.min())

In [141]:
# find median absolute error
median_absolute_error(y_test,y_pred)

0.2475937752746636

In [142]:
# find root mean square log error
np.sqrt(mean_squared_log_error(y_test,y_pred))

0.2236055865817873

In [143]:
# find r2 score
r2_score(y_test,y_pred)

-11.111931767657907

# ===============================================================

### Random Forest

In [144]:
# create a random forest regressor
rf = RandomForestRegressor()

In [145]:
# find optimal parameters with grid search
param_grid = {
'bootstrap': [True, False],
 'max_depth': [None,10, 20, 50, 100],
 'max_features': ['auto', 'sqrt'],
 'min_samples_leaf': [1, 2, 4],
 'min_samples_split': [2, 5, 10],
 'n_estimators': [100,200,500, 1000]
             }
grid = GridSearchCV(estimator=rf, param_grid=param_grid,
                    scoring=['neg_median_absolute_error','neg_mean_squared_log_error','r2'], refit = 'neg_mean_squared_log_error',
                    verbose=1, n_jobs=-1)

In [ ]:
# fit the grid on the train data
grid_result = grid.fit(X_train, y_train)

Fitting 5 folds for each of 720 candidates, totalling 3600 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 2 concurrent workers.
[Parallel(n_jobs=-1)]: Done  46 tasks      | elapsed:  2.2min
[Parallel(n_jobs=-1)]: Done 196 tasks      | elapsed:  9.0min
[Parallel(n_jobs=-1)]: Done 446 tasks      | elapsed: 17.3min
[Parallel(n_jobs=-1)]: Done 796 tasks      | elapsed: 28.7min
[Parallel(n_jobs=-1)]: Done 1246 tasks      | elapsed: 47.0min
[Parallel(n_jobs=-1)]: Done 1796 tasks      | elapsed: 67.6min
[Parallel(n_jobs=-1)]: Done 2446 tasks      | elapsed: 99.5min
[Parallel(n_jobs=-1)]: Done 3196 tasks      | elapsed: 140.0min
[Parallel(n_jobs=-1)]: Done 3600 out of 3600 | elapsed: 163.1min finished


In [ ]:
# print the best score and parameters found
print('Best Score: ', grid_result.best_score_)
print('Best Params: ', grid_result.best_params_)

Best Score:  -0.004418392261406096
Best Params:  {'bootstrap': True, 'max_depth': 10, 'max_features': 'sqrt', 'min_samples_leaf': 4, 'min_samples_split': 10, 'n_estimators': 200}


In [ ]:
# find predictions for test data
y_pred = grid_result.best_estimator_.predict(X_test)

In [ ]:
#normalizing y_pred
y_pred = (y_pred - y_pred.min())/(y_pred.max() - y_pred.min())

In [ ]:
# find median absolute error
median_absolute_error(y_test,y_pred)

0.2872661248004256

In [ ]:
# find root mean square log error
np.sqrt(mean_squared_log_error(y_test,y_pred))

0.2101381173298768

In [ ]:
# find r2 score
r2_score(y_test,y_pred)

-9.331726982797875

# ===============================================================

### SGRegression

In [146]:
# create a linear regressor
lr = SGDRegressor()

In [147]:
# find optimal parameters with grid search
param_grid = {
    
    'loss' : ['squared_loss','huber'],
    'alpha': [0.0001,0.001,0.01,0.1],
     'eta0': [0.01,0.1,1,10,100],
    'tol' : [0.00001,0.0001,0.001,0.01,0.1]    
 }
grid = GridSearchCV(estimator=lr, param_grid=param_grid,
                    scoring=['neg_median_absolute_error','r2'], refit = 'neg_median_absolute_error',
                    verbose=1, n_jobs=-1)

In [148]:
# fit the grid on the train data
grid_result = grid.fit(X_train, y_train)

Fitting 5 folds for each of 200 candidates, totalling 1000 fits


In [149]:
# print the best score and parameters found
print('Best Score: ', grid_result.best_score_)
print('Best Params: ', grid_result.best_params_)

Best Score:  -0.03492612920285247
Best Params:  {'alpha': 0.001, 'eta0': 0.1, 'loss': 'huber', 'tol': 0.1}


In [150]:
# find predictions for test data
y_pred = grid_result.best_estimator_.predict(X_test)

In [151]:
#normalizing y_pred
y_pred = (y_pred - y_pred.min())/(y_pred.max() - y_pred.min())

In [152]:
# find median absolute error
median_absolute_error(y_test,y_pred)

0.2486989779219191

In [153]:
# find root mean square log error
np.sqrt(mean_squared_log_error(y_test,y_pred))

0.2241543323981015

In [154]:
# find r2 score
r2_score(y_test,y_pred)

-11.1807894557034

# ===============================================================

### Decision Tree Regressor

In [155]:
dt = DecisionTreeRegressor()

In [156]:
# find optimal parameters with grid search
param_grid = {
    
    'criterion' : ['mse','mae'],
    'max_depth': range(10,20),
     'min_samples_split': range(2,10),
    'min_samples_leaf' : range(1,5)    
 }
grid = GridSearchCV(estimator=dt, param_grid=param_grid, cv=10,
                    scoring=['neg_median_absolute_error','neg_mean_squared_log_error','r2'], refit = 'neg_mean_squared_log_error',
                    verbose=1, n_jobs=-1)

In [157]:
# fit the grid on the train data
grid_result = grid.fit(X_train, y_train)

Fitting 10 folds for each of 640 candidates, totalling 6400 fits


In [158]:
# print the best score and parameters found
print('Best Score: ', grid_result.best_score_)
print('Best Params: ', grid_result.best_params_)

Best Score:  -0.00484721745060179
Best Params:  {'criterion': 'mae', 'max_depth': 10, 'min_samples_leaf': 4, 'min_samples_split': 7}


In [159]:
# find predictions for test data
y_pred = grid_result.best_estimator_.predict(X_test)

In [160]:
#normalizing y_pred
y_pred = (y_pred - y_pred.min())/(y_pred.max() - y_pred.min())

In [161]:
# find median absolute error
median_absolute_error(y_test,y_pred)

0.2239422598307616

In [162]:
# find root mean square log error
np.sqrt(mean_squared_log_error(y_test,y_pred))

0.18900328356148613

In [163]:
# find r2 score
r2_score(y_test,y_pred)

-6.969489312348588

# ===============================================================

### XGBoost Regressor

In [164]:
xgbtree = XGBRegressor()

In [165]:
# find optimal parameters with grid search
param_grid = {
              'learning_rate': [0.001,0.01,0.1,1],
              'max_depth': range(2,10),
              'subsample':[0.01,0.1,1] ,
              'colsample_bytree': [0.01,0.1,1],
              'n_estimators': [100,200,500,1000]}
grid = GridSearchCV(estimator=xgbtree, param_grid=param_grid, cv=5,
                    scoring=['neg_median_absolute_error','r2'], refit = 'neg_median_absolute_error',
                    verbose=1, n_jobs=-1)

In [ ]:
# fit the grid on the train data
grid_result = grid.fit(X_train, y_train)

Fitting 5 folds for each of 1152 candidates, totalling 5760 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 2 concurrent workers.
[Parallel(n_jobs=-1)]: Done  46 tasks      | elapsed:   13.4s
[Parallel(n_jobs=-1)]: Done 196 tasks      | elapsed:  1.5min
[Parallel(n_jobs=-1)]: Done 446 tasks      | elapsed:  4.0min
[Parallel(n_jobs=-1)]: Done 796 tasks      | elapsed:  7.6min
[Parallel(n_jobs=-1)]: Done 1246 tasks      | elapsed: 12.6min
[Parallel(n_jobs=-1)]: Done 1796 tasks      | elapsed: 19.3min
[Parallel(n_jobs=-1)]: Done 2446 tasks      | elapsed: 26.0min
[Parallel(n_jobs=-1)]: Done 3196 tasks      | elapsed: 34.2min
[Parallel(n_jobs=-1)]: Done 4046 tasks      | elapsed: 44.0min
[Parallel(n_jobs=-1)]: Done 4996 tasks      | elapsed: 56.7min
[Parallel(n_jobs=-1)]: Done 5760 out of 5760 | elapsed: 67.7min finished


[23:15:38] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.


In [166]:
# print the best score and parameters found
print('Best Score: ', grid_result.best_score_)
print('Best Params: ', grid_result.best_params_)

Best Score:  -0.00484721745060179
Best Params:  {'criterion': 'mae', 'max_depth': 10, 'min_samples_leaf': 4, 'min_samples_split': 7}


In [167]:
# find predictions for test data
y_pred = grid_result.best_estimator_.predict(X_test)

In [168]:
#normalizing y_pred
y_pred = (y_pred - y_pred.min())/(y_pred.max() - y_pred.min())

In [169]:
# find median absolute error
median_absolute_error(y_test,y_pred)

0.2239422598307616

In [170]:
# find root mean square log error
np.sqrt(mean_squared_log_error(y_test,y_pred))

0.18900328356148613

In [171]:
# find r2 score
r2_score(y_test,y_pred)

-6.969489312348588

# ===============================================================

### AdaBoost Regressor

In [172]:
abreg = AdaBoostRegressor(base_estimator=DecisionTreeRegressor(),
                              random_state=0)

In [ ]:
# find optimal parameters with grid search
param_grid = { 'loss' : ['linear', 'square', 'exponential'],
              'learning_rate': [0.001,0.01,0.1,1],
              'base_estimator__max_depth': range(2,10),
              'n_estimators': [100,200,500,1000]}
grid = GridSearchCV(estimator=abreg, param_grid=param_grid, cv=5,
                    scoring=['neg_median_absolute_error','r2'], refit = 'neg_median_absolute_error',
                    verbose=1, n_jobs=-1)

In [ ]:
# fit the grid on the train data
grid_result = grid.fit(X_train, y_train)

Fitting 5 folds for each of 384 candidates, totalling 1920 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 2 concurrent workers.
[Parallel(n_jobs=-1)]: Done  46 tasks      | elapsed:  1.2min
[Parallel(n_jobs=-1)]: Done 196 tasks      | elapsed:  4.1min
[Parallel(n_jobs=-1)]: Done 446 tasks      | elapsed:  9.7min
[Parallel(n_jobs=-1)]: Done 796 tasks      | elapsed: 20.4min
[Parallel(n_jobs=-1)]: Done 1246 tasks      | elapsed: 35.2min
[Parallel(n_jobs=-1)]: Done 1796 tasks      | elapsed: 58.1min
[Parallel(n_jobs=-1)]: Done 1920 out of 1920 | elapsed: 61.6min finished


In [ ]:
# print the best score and parameters found
print('Best Score: ', grid_result.best_score_)
print('Best Params: ', grid_result.best_params_)

Best Score:  -0.0347410677510596
Best Params:  {'base_estimator__max_depth': 9, 'learning_rate': 0.001, 'loss': 'linear', 'n_estimators': 100}


In [ ]:
# find predictions for test data
y_pred = grid_result.best_estimator_.predict(X_test)

In [ ]:
#normalizing y_pred
y_pred = (y_pred - y_pred.min())/(y_pred.max() - y_pred.min())

In [ ]:
# find median absolute error
median_absolute_error(y_test,y_pred)

0.19729867696620906

In [ ]:
# find root mean square log error
np.sqrt(mean_squared_log_error(y_test,y_pred))

0.1536971298773078

In [ ]:
# find r2 score
r2_score(y_test,y_pred)

-4.110286046096933

# ===============================================================

### MLP Regressor

In [173]:
from tensorflow.keras.callbacks import ModelCheckpoint, TensorBoard

In [174]:
import datetime

In [175]:
# Creating a Neural Network Model
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Activation, Dropout
from tensorflow.keras.optimizers import Adam

In [176]:
model = Sequential()
model.add(Dense(1200,activation='relu'))
model.add(Dropout(0.2))
model.add(Dense(1200,activation='relu'))
model.add(Dropout(0.2))
model.add(Dense(128,activation='relu'))
model.add(Dropout(0.2))
model.add(Dense(128,activation='relu'))
model.add(Dropout(0.2))
model.add(Dense(128,activation='relu'))
model.add(Dropout(0.2))
model.add(Dense(1))

In [177]:
import tensorflow.keras.backend as K

In [178]:

def rmsle(y_true, y_pred):
    msle = tf.keras.losses.MeanSquaredLogarithmicError()
    return K.sqrt(msle(y_true, y_pred)) 

In [179]:
model.compile(optimizer='Adam',loss=rmsle,metrics=[tf.keras.metrics.RootMeanSquaredError()])

In [180]:
model.fit(x=X_train,y=y_train,
          validation_data=(X_test,y_test),
          batch_size=64,epochs=100)


Epoch 1/100
103/103 [==============================] - 7s 51ms/step - loss: 0.0961 - root_mean_squared_error: 0.1266 - val_loss: 0.0658 - val_root_mean_squared_error: 0.0859
Epoch 2/100
103/103 [==============================] - 4s 42ms/step - loss: 0.0671 - root_mean_squared_error: 0.0869 - val_loss: 0.0654 - val_root_mean_squared_error: 0.0856
Epoch 3/100
103/103 [==============================] - 5s 48ms/step - loss: 0.0696 - root_mean_squared_error: 0.0914 - val_loss: 0.0654 - val_root_mean_squared_error: 0.0859
Epoch 4/100
103/103 [==============================] - 4s 42ms/step - loss: 0.0686 - root_mean_squared_error: 0.0899 - val_loss: 0.0655 - val_root_mean_squared_error: 0.0856
Epoch 5/100
103/103 [==============================] - 4s 39ms/step - loss: 0.0671 - root_mean_squared_error: 0.0875 - val_loss: 0.0663 - val_root_mean_squared_error: 0.0871
Epoch 6/100
103/103 [==============================] - 4s 39ms/step - loss: 0.0677 - root_mean_squared_error: 0.0877 - val_loss: 0

103/103 [==============================] - 3s 30ms/step - loss: 0.0659 - root_mean_squared_error: 0.0861 - val_loss: 0.0655 - val_root_mean_squared_error: 0.0860
Epoch 94/100
103/103 [==============================] - 3s 32ms/step - loss: 0.0636 - root_mean_squared_error: 0.0832 - val_loss: 0.0659 - val_root_mean_squared_error: 0.0866
Epoch 95/100
103/103 [==============================] - 3s 28ms/step - loss: 0.0641 - root_mean_squared_error: 0.0837 - val_loss: 0.0656 - val_root_mean_squared_error: 0.0863
Epoch 96/100
103/103 [==============================] - 3s 28ms/step - loss: 0.0665 - root_mean_squared_error: 0.0868 - val_loss: 0.0656 - val_root_mean_squared_error: 0.0860
Epoch 97/100
103/103 [==============================] - 3s 29ms/step - loss: 0.0667 - root_mean_squared_error: 0.0878 - val_loss: 0.0658 - val_root_mean_squared_error: 0.0862
Epoch 98/100
103/103 [==============================] - 3s 28ms/step - loss: 0.0659 - root_mean_squared_error: 0.0860 - val_loss: 0.0657 -

In [181]:
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense (Dense)                (None, 1200)              3600      
_________________________________________________________________
dropout (Dropout)            (None, 1200)              0         
_________________________________________________________________
dense_1 (Dense)              (None, 1200)              1441200   
_________________________________________________________________
dropout_1 (Dropout)          (None, 1200)              0         
_________________________________________________________________
dense_2 (Dense)              (None, 128)               153728    
_________________________________________________________________
dropout_2 (Dropout)          (None, 128)               0         
_________________________________________________________________
dense_3 (Dense)              (None, 128)               1

In [182]:
K.clear_session()

In [183]:
y_pred = model.predict(X_test)

In [184]:
y_pred.shape

(1644, 1)

In [185]:
# find median absolute error
median_absolute_error(y_test,y_pred)

0.034375390010636026

In [186]:
# find root mean square log error
np.sqrt(mean_squared_log_error(y_test,y_pred))

0.06645822429299328

In [187]:
# find r2 score
r2_score(y_test,y_pred)

0.10452870396800462

# ===============================================================

# =============================================================
# =============================================================
## ============================= Springs =====================================
# =============================================================
# =============================================================

In [188]:
springs.head()

,Mean_Rainfall,Mean_Temp,Actual_Depth,Actual_Flow_Rate,Date
0,4.125714,-0.084921,-131.779048,-7.393259,2000-01-01
1,4.737143,6.701587,-132.510476,-7.393882,2000-01-02
2,6.824762,5.405556,-130.986190,-7.597644,2000-01-03
3,5.326667,4.813492,-129.141111,-6.168792,2000-01-04
4,4.177143,14.011905,-130.089524,-7.344170,2000-01-05


In [189]:
# Normalizing the columns to remove negative values
springs['Mean_Rainfall'] = (springs['Mean_Rainfall'] - springs['Mean_Rainfall'].min()) / (springs['Mean_Rainfall'].max() - springs['Mean_Rainfall'].min())
springs['Mean_Temp'] = (springs['Mean_Temp'] - springs['Mean_Temp'].min()) / (springs['Mean_Temp'].max() - springs['Mean_Temp'].min())
springs['Actual_Depth'] = (springs['Actual_Depth'] - springs['Actual_Depth'].min()) / (springs['Actual_Depth'].max() - springs['Actual_Depth'].min())
springs['Actual_Flow_Rate'] = (springs['Actual_Flow_Rate'] - springs['Actual_Flow_Rate'].min()) / (springs['Actual_Flow_Rate'].max() - springs['Actual_Flow_Rate'].min())

In [190]:
springs.head()

,Mean_Rainfall,Mean_Temp,Actual_Depth,Actual_Flow_Rate,Date
0,0.076309,0.169289,0.416992,0.530379,2000-01-01
1,0.087618,0.357043,0.346262,0.530375,2000-01-02
2,0.126230,0.321187,0.493661,0.529121,2000-01-03
3,0.098522,0.304807,0.672080,0.537916,2000-01-04
4,0.077260,0.559289,0.580368,0.530681,2000-01-05


In [191]:
# separating the date and target feature
X = springs.drop(['Date','Actual_Flow_Rate'],axis=1)
y = springs['Actual_Flow_Rate']

In [192]:
#Dividing the dataset into train and test for features as well as labels
X_train, X_test, y_train, y_test = train_test_split(X,y,test_size=0.2,random_state=42)

In [193]:
#checking the number of rows in training data
len(X_train)

5989

In [194]:
#checking the number of rows in training labels
len(y_train)

5989

In [195]:
#checking the number of rows in testing data
len(X_test)

1498

In [196]:
#checking the number of rows in testing labels
len(y_test)

1498

# =============================================================

### knn

In [197]:
# create a knn regressor
neigh = KNeighborsRegressor()

In [198]:
# find optimal parameters with grid search
param_grid = {
    'n_neighbors': [3,5,7,9,11,19],
    'weights': ['uniform', 'distance'],
    'metric' : ['euclidean','manhattan']
}
grid = GridSearchCV(estimator=neigh, param_grid=param_grid,
                    scoring=['neg_median_absolute_error','neg_mean_squared_log_error','r2'], refit = 'neg_mean_squared_log_error',
                    verbose=1, n_jobs=-1)

In [199]:
# fit the grid on the train data
grid_result = grid.fit(X_train, y_train)

Fitting 5 folds for each of 24 candidates, totalling 120 fits


In [200]:
# print the best score and parameters found
print('Best Score: ', grid_result.best_score_)
print('Best Params: ', grid_result.best_params_)

Best Score:  -0.021191998046753598
Best Params:  {'metric': 'manhattan', 'n_neighbors': 19, 'weights': 'distance'}


In [202]:
# find predictions for test data
y_pred = grid_result.best_estimator_.predict(X_test)

In [203]:
#normalizing y_pred
y_pred = (y_pred - y_pred.min())/(y_pred.max() - y_pred.min())

In [204]:
# find median absolute error
median_absolute_error(y_test,y_pred)

0.16653555878093784

In [205]:
# find root mean square log error
np.sqrt(mean_squared_log_error(y_test,y_pred))

0.16545173015424464

In [206]:
# find r2 score
r2_score(y_test,y_pred)

-0.1472021736660043

# =============================================================

### Multiple Linear Regression

In [207]:
# create a linear regressor
lr = LinearRegression()

In [208]:
# fit the lr on train data
lr = lr.fit(X_train, y_train)

In [209]:
# find the predictions
y_pred = lr.predict(X_test)

In [210]:
#normalizing y_pred
y_pred = (y_pred - y_pred.min())/(y_pred.max() - y_pred.min())

In [211]:
# find median absolute error
median_absolute_error(y_test,y_pred)

0.3106472486430929

In [212]:
# find root mean square log error
np.sqrt(mean_squared_log_error(y_test,y_pred))

0.2848226752175971

In [213]:
# find r2 score
r2_score(y_test,y_pred)

-2.1411975666849132

R2  is negative only when the chosen model does not follow the trend of the data, so fits worse than a horizontal line.

# =============================================================

### Random Forest

In [ ]:
# create a random forest regressor
rf = RandomForestRegressor()

In [ ]:
# find optimal parameters with grid search
param_grid = {
'bootstrap': [True, False],
 'max_depth': [None,10, 20, 50, 100],
 'max_features': ['auto', 'sqrt'],
 'min_samples_leaf': [1, 2, 4],
 'min_samples_split': [2, 5, 10],
 'n_estimators': [100,200,500, 1000]
             }
grid = GridSearchCV(estimator=rf, param_grid=param_grid,
                    scoring=['neg_median_absolute_error','neg_mean_squared_log_error','r2'], refit = 'neg_mean_squared_log_error',
                    verbose=1, n_jobs=-1)

In [ ]:
# fit the grid on the train data
grid_result = grid.fit(X_train, y_train)

Fitting 5 folds for each of 720 candidates, totalling 3600 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 2 concurrent workers.
[Parallel(n_jobs=-1)]: Done  46 tasks      | elapsed:  3.6min
[Parallel(n_jobs=-1)]: Done 196 tasks      | elapsed: 14.4min
[Parallel(n_jobs=-1)]: Done 446 tasks      | elapsed: 25.3min
[Parallel(n_jobs=-1)]: Done 796 tasks      | elapsed: 41.6min
[Parallel(n_jobs=-1)]: Done 1246 tasks      | elapsed: 68.6min
[Parallel(n_jobs=-1)]: Done 1796 tasks      | elapsed: 97.1min
[Parallel(n_jobs=-1)]: Done 2446 tasks      | elapsed: 147.2min
[Parallel(n_jobs=-1)]: Done 3196 tasks      | elapsed: 209.0min
[Parallel(n_jobs=-1)]: Done 3600 out of 3600 | elapsed: 242.5min finished


In [ ]:
# print the best score and parameters found
print('Best Score: ', grid_result.best_score_)
print('Best Params: ', grid_result.best_params_)

Best Score:  -0.016208164634010154
Best Params:  {'bootstrap': True, 'max_depth': None, 'max_features': 'sqrt', 'min_samples_leaf': 1, 'min_samples_split': 2, 'n_estimators': 500}


In [ ]:
# find predictions for test data
y_pred = grid_result.best_estimator_.predict(X_test)

In [ ]:
#normalizing y_pred
y_pred = (y_pred - y_pred.min())/(y_pred.max() - y_pred.min())

In [ ]:
# find median absolute error
median_absolute_error(y_test,y_pred)

0.06339626104495508

In [ ]:
# find root mean square log error
np.sqrt(mean_squared_log_error(y_test,y_pred))

0.12466759306708458

In [ ]:
# find r2 score
r2_score(y_test,y_pred)

0.3552887850735922

# =============================================================

### SGRegression

In [214]:
# create a linear regressor
lr = SGDRegressor()

In [215]:
# find optimal parameters with grid search
param_grid = {
    
    'loss' : ['squared_loss','huber'],
    'alpha': [0.0001,0.001,0.01,0.1],
     'eta0': [0.01,0.1,1,10,100],
    'tol' : [0.00001,0.0001,0.001,0.01,0.1]    
 }
grid = GridSearchCV(estimator=lr, param_grid=param_grid,
                    scoring=['neg_median_absolute_error','r2'], refit = 'neg_median_absolute_error',
                    verbose=1, n_jobs=-1)

In [216]:
# fit the grid on the train data
grid_result = grid.fit(X_train, y_train)

Fitting 5 folds for each of 200 candidates, totalling 1000 fits


In [217]:
# print the best score and parameters found
print('Best Score: ', grid_result.best_score_)
print('Best Params: ', grid_result.best_params_)

Best Score:  -0.19886521836378318
Best Params:  {'alpha': 0.0001, 'eta0': 10, 'loss': 'squared_loss', 'tol': 0.1}


In [218]:
# find predictions for test data
y_pred = grid_result.best_estimator_.predict(X_test)

In [219]:
#normalizing y_pred
y_pred = (y_pred - y_pred.min())/(y_pred.max() - y_pred.min())

In [220]:
# find median absolute error
median_absolute_error(y_test,y_pred)

0.1578245789394696

In [221]:
# find root mean square log error
np.sqrt(mean_squared_log_error(y_test,y_pred))

0.2169134895048725

In [222]:
# find r2 score
r2_score(y_test,y_pred)

-0.8937296652360205

# =============================================================

### Decision Tree Regressor

In [223]:
dt = DecisionTreeRegressor()

In [224]:
# find optimal parameters with grid search
param_grid = {
    
    'criterion' : ['mse','mae'],
    'max_depth': range(10,20),
     'min_samples_split': range(2,10),
    'min_samples_leaf' : range(1,5)    
 }
grid = GridSearchCV(estimator=dt, param_grid=param_grid, cv=10,
                    scoring=['neg_median_absolute_error','neg_mean_squared_log_error','r2'], refit = 'neg_mean_squared_log_error',
                    verbose=1, n_jobs=-1)

In [225]:
# fit the grid on the train data
grid_result = grid.fit(X_train, y_train)

Fitting 10 folds for each of 640 candidates, totalling 6400 fits


In [232]:
# print the best score and parameters found
print('Best Score: ', grid_result.best_score_)
print('Best Params: ', grid_result.best_params_)

Best Score:  -0.021627997495911627
Best Params:  {'criterion': 'mse', 'max_depth': 10, 'min_samples_leaf': 2, 'min_samples_split': 9}


In [233]:
# find predictions for test data
y_pred = grid_result.best_estimator_.predict(X_test)

In [234]:
#normalizing y_pred
y_pred = (y_pred - y_pred.min())/(y_pred.max() - y_pred.min())

In [235]:
# find median absolute error
median_absolute_error(y_test,y_pred)

0.08793477057526328

In [236]:
# find root mean square log error
np.sqrt(mean_squared_log_error(y_test,y_pred))

0.13827971812096929

In [237]:
# find r2 score
r2_score(y_test,y_pred)

0.21134319535530133

# =============================================================

### XGBoost Regressor

In [238]:
xgbtree = XGBRegressor()

In [239]:
# find optimal parameters with grid search
param_grid = {
              'learning_rate': [0.001,0.01,0.1,1],
              'max_depth': range(2,10),
              'subsample':[0.01,0.1,1] ,
              'colsample_bytree': [0.01,0.1,1],
              'n_estimators': [100,200,500,1000]}
grid = GridSearchCV(estimator=xgbtree, param_grid=param_grid, cv=5,
                    scoring=['neg_median_absolute_error','r2'], refit = 'neg_median_absolute_error',
                    verbose=1, n_jobs=-1)

In [ ]:
# fit the grid on the train data
grid_result = grid.fit(X_train, y_train)

Fitting 5 folds for each of 1152 candidates, totalling 5760 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 2 concurrent workers.
[Parallel(n_jobs=-1)]: Done  46 tasks      | elapsed:   13.1s
[Parallel(n_jobs=-1)]: Done 196 tasks      | elapsed:  1.5min
[Parallel(n_jobs=-1)]: Done 446 tasks      | elapsed:  4.6min
[Parallel(n_jobs=-1)]: Done 796 tasks      | elapsed:  8.3min
[Parallel(n_jobs=-1)]: Done 1246 tasks      | elapsed: 13.3min
[Parallel(n_jobs=-1)]: Done 1796 tasks      | elapsed: 19.7min
[Parallel(n_jobs=-1)]: Done 2446 tasks      | elapsed: 27.1min
[Parallel(n_jobs=-1)]: Done 3196 tasks      | elapsed: 35.2min
[Parallel(n_jobs=-1)]: Done 4046 tasks      | elapsed: 44.9min
[Parallel(n_jobs=-1)]: Done 4996 tasks      | elapsed: 60.7min
[Parallel(n_jobs=-1)]: Done 5760 out of 5760 | elapsed: 73.3min finished


[02:10:46] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.


In [ ]:
# print the best score and parameters found
print('Best Score: ', grid_result.best_score_)
print('Best Params: ', grid_result.best_params_)

Best Score:  -0.07022984297011255
Best Params:  {'colsample_bytree': 1, 'learning_rate': 0.1, 'max_depth': 9, 'n_estimators': 1000, 'subsample': 1}


In [ ]:
# find predictions for test data
y_pred = grid_result.best_estimator_.predict(X_test)

In [ ]:
#normalizing y_pred
y_pred = (y_pred - y_pred.min())/(y_pred.max() - y_pred.min())

In [ ]:
# find median absolute error
median_absolute_error(y_test,y_pred)

0.07750636324936641

In [ ]:
# find root mean square log error
np.sqrt(mean_squared_log_error(y_test,y_pred))

0.12503603225652735

In [ ]:
# find r2 score
r2_score(y_test,y_pred)

0.3485884002530737

# =============================================================

### AdaBoost Regressor

In [ ]:
abreg = AdaBoostRegressor(base_estimator=DecisionTreeRegressor(),
                              random_state=0)

In [ ]:
# find optimal parameters with grid search
param_grid = { 'loss' : ['linear', 'square', 'exponential'],
              'learning_rate': [0.001,0.01,0.1,1],
              'base_estimator__max_depth': range(2,10),
              'n_estimators': [100,200,500,1000]}
grid = GridSearchCV(estimator=abreg, param_grid=param_grid, cv=5,
                    scoring=['neg_median_absolute_error','r2'], refit = 'neg_median_absolute_error',
                    verbose=1, n_jobs=-1)

In [ ]:
# fit the grid on the train data
grid_result = grid.fit(X_train, y_train)

Fitting 5 folds for each of 384 candidates, totalling 1920 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 2 concurrent workers.
[Parallel(n_jobs=-1)]: Done  46 tasks      | elapsed:  1.5min
[Parallel(n_jobs=-1)]: Done 196 tasks      | elapsed:  4.6min
[Parallel(n_jobs=-1)]: Done 446 tasks      | elapsed: 10.9min
[Parallel(n_jobs=-1)]: Done 796 tasks      | elapsed: 25.5min
[Parallel(n_jobs=-1)]: Done 1246 tasks      | elapsed: 48.1min
[Parallel(n_jobs=-1)]: Done 1796 tasks      | elapsed: 86.3min
[Parallel(n_jobs=-1)]: Done 1920 out of 1920 | elapsed: 95.1min finished


In [ ]:
# print the best score and parameters found
print('Best Score: ', grid_result.best_score_)
print('Best Params: ', grid_result.best_params_)

Best Score:  -0.10607506721892213
Best Params:  {'base_estimator__max_depth': 9, 'learning_rate': 0.001, 'loss': 'linear', 'n_estimators': 100}


In [ ]:
# find predictions for test data
y_pred = grid_result.best_estimator_.predict(X_test)

In [ ]:
#normalizing y_pred
y_pred = (y_pred - y_pred.min())/(y_pred.max() - y_pred.min())

In [ ]:
# find median absolute error
median_absolute_error(y_test,y_pred)

0.09586101757730897

In [ ]:
# find root mean square log error
np.sqrt(mean_squared_log_error(y_test,y_pred))

0.13071360759079187

In [ ]:
# find r2 score
r2_score(y_test,y_pred)

0.30911404242888374

# =============================================================

### MLP Regressor

In [240]:
from tensorflow.keras.callbacks import ModelCheckpoint, TensorBoard

In [241]:
# Creating a Neural Network Model
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Activation, Dropout
from tensorflow.keras.optimizers import Adam

In [242]:
model = Sequential()
model.add(Dense(512,activation='relu'))
model.add(Dropout(0.2))
model.add(Dense(512,activation='relu'))
model.add(Dropout(0.2))
model.add(Dense(128,activation='relu'))
model.add(Dropout(0.2))
model.add(Dense(128,activation='relu'))
model.add(Dropout(0.2))
model.add(Dense(128,activation='relu'))
model.add(Dropout(0.2))
model.add(Dense(1))

In [243]:
import tensorflow.keras.backend as K

In [244]:

def rmsle(y_true, y_pred):
    msle = tf.keras.losses.MeanSquaredLogarithmicError()
    return K.sqrt(msle(y_true, y_pred)) 

In [245]:
model.compile(optimizer='Adam',loss=rmsle,metrics=[tf.keras.metrics.RootMeanSquaredError()])

In [246]:
model.fit(x=X_train,y=y_train,
          validation_data=(X_test,y_test),
          batch_size=64,epochs=100)


Epoch 1/100
94/94 [==============================] - 2s 12ms/step - loss: 0.2280 - root_mean_squared_error: 0.3701 - val_loss: 0.1505 - val_root_mean_squared_error: 0.2420
Epoch 2/100
94/94 [==============================] - 1s 10ms/step - loss: 0.1616 - root_mean_squared_error: 0.2595 - val_loss: 0.1503 - val_root_mean_squared_error: 0.2415
Epoch 3/100
94/94 [==============================] - 1s 9ms/step - loss: 0.1572 - root_mean_squared_error: 0.2533 - val_loss: 0.1512 - val_root_mean_squared_error: 0.2461
Epoch 4/100
94/94 [==============================] - 1s 10ms/step - loss: 0.1601 - root_mean_squared_error: 0.2570 - val_loss: 0.1509 - val_root_mean_squared_error: 0.2455
Epoch 5/100
94/94 [==============================] - 1s 10ms/step - loss: 0.1579 - root_mean_squared_error: 0.2539 - val_loss: 0.1496 - val_root_mean_squared_error: 0.2422
Epoch 6/100
94/94 [==============================] - 1s 9ms/step - loss: 0.1598 - root_mean_squared_error: 0.2554 - val_loss: 0.1478 - val_ro

94/94 [==============================] - 1s 13ms/step - loss: 0.1439 - root_mean_squared_error: 0.2301 - val_loss: 0.1413 - val_root_mean_squared_error: 0.2268
Epoch 96/100
94/94 [==============================] - 1s 12ms/step - loss: 0.1417 - root_mean_squared_error: 0.2268 - val_loss: 0.1418 - val_root_mean_squared_error: 0.2292
Epoch 97/100
94/94 [==============================] - 1s 11ms/step - loss: 0.1449 - root_mean_squared_error: 0.2311 - val_loss: 0.1397 - val_root_mean_squared_error: 0.2250
Epoch 98/100
94/94 [==============================] - 1s 10ms/step - loss: 0.1408 - root_mean_squared_error: 0.2248 - val_loss: 0.1408 - val_root_mean_squared_error: 0.2283
Epoch 99/100
94/94 [==============================] - 1s 10ms/step - loss: 0.1422 - root_mean_squared_error: 0.2279 - val_loss: 0.1408 - val_root_mean_squared_error: 0.2275
Epoch 100/100
94/94 [==============================] - 1s 10ms/step - loss: 0.1432 - root_mean_squared_error: 0.2283 - val_loss: 0.1451 - val_root_m

In [ ]:
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense (Dense)                (None, 512)               2048      
_________________________________________________________________
dropout (Dropout)            (None, 512)               0         
_________________________________________________________________
dense_1 (Dense)              (None, 512)               262656    
_________________________________________________________________
dropout_1 (Dropout)          (None, 512)               0         
_________________________________________________________________
dense_2 (Dense)              (None, 128)               65664     
_________________________________________________________________
dropout_2 (Dropout)          (None, 128)               0         
_________________________________________________________________
dense_3 (Dense)              (None, 128)               1

In [ ]:
K.clear_session()

In [ ]:
y_pred = model.predict(X_test)

In [ ]:
y_pred.shape

(1498, 1)

In [ ]:
# find median absolute error
median_absolute_error(y_test,y_pred)

0.1378374286099205

In [ ]:
# find root mean square log error
np.sqrt(mean_squared_log_error(y_test,y_pred))

0.14250678100449088

In [ ]:
# find r2 score
r2_score(y_test,y_pred)

0.14410067815385863

# =============================================================

# Summary

In [247]:
from prettytable import PrettyTable

In [248]:
# Table for summarizing water body type and their R2 score with different models
finalTable = PrettyTable(["Model","Aquifers", "Lakes", "Rivers", "Springs"])
  
# Add rows
finalTable.add_row(["knn", "0.96", "-17.851,-1.254", "-5.839", "-0.147"])
finalTable.add_row(["Linear Regression", "-0.361", "-156.95,-0.821", "-11.11", "-2.141"])
finalTable.add_row(["Random Forest", "0.966", "-25.89,-0.684", "-9.331", "0.355"])
finalTable.add_row(["SGRegression", "-0.171", "-0.781", "-11.21", "-1.11"])
finalTable.add_row(["Decision Tree", "0.956", "-0.091", "-7.012", "0.211"])
finalTable.add_row(["XGBoost", "0.958", "-0.325", "-11.698", "0.348"])
finalTable.add_row(["AdaBoost", "0.964", "-0.053", "-4.11", "0.309"])
finalTable.add_row(["MLP", "0.958","0.076", "0.106", "0.181"])
  
print(finalTable)

+-------------------+----------+----------------+---------+---------+
|       Model       | Aquifers |     Lakes      |  Rivers | Springs |
+-------------------+----------+----------------+---------+---------+
|        knn        |   0.96   | -17.851,-1.254 |  -5.839 |  -0.147 |
| Linear Regression |  -0.361  | -156.95,-0.821 |  -11.11 |  -2.141 |
|   Random Forest   |  0.966   | -25.89,-0.684  |  -9.331 |  0.355  |
|    SGRegression   |  -0.171  |     -0.781     |  -11.21 |  -1.11  |
|   Decision Tree   |  0.956   |     -0.091     |  -7.012 |  0.211  |
|      XGBoost      |  0.958   |     -0.325     | -11.698 |  0.348  |
|      AdaBoost     |  0.964   |     -0.053     |  -4.11  |  0.309  |
|        MLP        |  0.958   |     0.076      |  0.106  |  0.181  |
+-------------------+----------+----------------+---------+---------+
